## Limpiar y Normalizar

In [11]:
import pandas as pd
import sqlite3
import re
from contextlib import closing
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [28]:
db = "test 2.db"

def get_uid(name:str)->str:
    return re.findall(r"\d+", name)

def get_date(name:int)->datetime:
    return datetime.strptime(
        re.findall(r"\d+", str(name))[0],
        r'%Y%m%d%H%M%S'
    ).strftime(r"%c")

def list_sessions():
    with closing(sqlite3.connect(db)) as conn:
        sessions = pd.read_sql(
            "SELECT id, notes FROM session WHERE (notes is not NULL) and (notes is not \"\") and (sensors == 5)", conn
            )
        sessions["date"] = sessions["id"].apply(get_date)

        return sessions

def get_session(uid:int):
    with closing(sqlite3.connect(db)) as conn:
        return pd.read_sql(
            f"SELECT * FROM session_{uid} WHERE (signal_strength0 == 0) and (signal_strength1 == 0) and (signal_strength2 == 0) and (signal_strength3 == 0) and (signal_strength4 == 0)", conn
        )

#def get_session(uid:int):
#    with closing(sqlite3.connect(db)) as conn:
#        return pd.read_sql(
#            f"SELECT * FROM session_{uid} WHERE ((signal_strength0 == 0) or (signal_strength0 is not NULL)) and ((signal_strength1 == 0) or (signal_strength1 is not NULL)) and ((signal_strength2 == 0) or (signal_strength2 is not NULL)) and ((signal_strength3 == 0) or (signal_strength3 is not NULL)) and ((signal_strength4 == 0) or (signal_strength4 is not NULL))", conn
#        )

In [29]:
sessions = list_sessions()
sessions

,id,notes,date
0,20240912144149,ivan \nevento 1 buena\nevento 2 mala,Thu Sep 12 14:41:49 2024
1,20240912154538,dana,Thu Sep 12 15:45:38 2024
2,20240912155255,dana,Thu Sep 12 15:52:55 2024
3,20240912160444,aidee,Thu Sep 12 16:04:44 2024
4,20240912161448,aidee,Thu Sep 12 16:14:48 2024
...,...,...,...
74,20241107145351,sofi 1,Thu Nov 7 14:53:51 2024
75,20241107150554,sofi 3,Thu Nov 7 15:05:54 2024
76,20241107151958,miranda5,Thu Nov 7 15:19:58 2024
77,20241130090030,Marcelo,Sat Nov 30 09:00:30 2024


In [46]:
sessions = list_sessions()
scaler = MinMaxScaler()
size = sessions.index.size
for i in range(size):
    with closing(sqlite3.connect(db)) as conn:
        session = get_session(sessions.iloc[i,0])
        try:
            print(i)
            z = scaler.fit_transform(session)
            df = pd.DataFrame(data=z, dtype=np.float32)
            print(df)
            df.to_sql(name='datos', con=conn , if_exists='replace')
        except:
            ...
        #print(z)

0
      0         1         2         3         4         5         6   \
0    0.0  0.597701  0.622222  0.216561  0.211706  0.102059  0.391508   
1    0.0  0.528736  0.555556  0.298774  0.176865  0.118880  0.602800   
2    0.0  0.563218  0.377778  0.196665  0.014782  0.014882  0.011894   
3    0.0  0.609195  0.133333  0.042723  0.036615  0.009812  0.037659   
4    0.0  0.597701  0.377778  0.198014  0.040428  0.066572  0.179567   
..   ...       ...       ...       ...       ...       ...       ...   
250  0.0  0.103448  0.733333  0.036989  0.044475  0.014585  0.111588   
251  0.0  0.103448  0.622222  0.446838  0.150373  0.261274  0.062997   
252  0.0  0.172414  0.655556  0.124072  0.065938  0.052534  0.235891   
253  0.0  0.459770  0.655556  0.596162  0.113867  0.153108  0.118872   
254  0.0  0.597701  0.622222  0.168496  0.050990  0.059638  0.058325   

           7         8         9   ...        45        46        47  \
0    0.151757  0.146250  0.149713  ...  0.544304  0.012987  0

In [44]:
df.shape

(23, 55)